In [ ]:
import torch
from sqlnet.utils import *
from sqlnet.model.sqlbert import SQLBert, BertTokenizer
from sqlnet.lib.dbengine import DBEngine
import argparse
import time
import os.path as osp
from flask import Flask
from flask import request, Flask, jsonify
from flask_cors import *
 
app = Flask(__name__)

data_dir = '../data/final/'
bert_model_dir = '../model/chinese-bert_chinese_wwm_pytorch/'
restore_model_path = '../model/saved_bert_model'
test_sql_path = osp.join(data_dir, 'final_test.json')
test_table_path = osp.join(data_dir, 'final_test.tables.json')
test_db_path = osp.join(data_dir,'final_test.db')
_, test_table = load_data(test_sql_path, test_table_path)
tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case=True)
model = SQLBert.from_pretrained(bert_model_dir)
model.load_state_dict(torch.load(restore_model_path, map_location=torch.device('cpu')))

def solve(question, table_id):
    return my_predict_test(model, [{"table_id": table_id, "question": question}], test_table,tokenizer=tokenizer)

def results(rst,table_id):
    engine = DBEngine(test_db_path)
    return engine.my_execute(table_id, rst['sel'], rst['agg'], rst['conds'], rst['cond_conn_op'])
    
@app.route('/nl2sql', methods=['POST'])
def post_Data():
    table_id = request.form['table_id']
    question = request.form['question']
    if (question.strip()=="" or table_id.strip()==""):
        res = {"msg": "table_id和查询语句不能为空！"}
    else:
        try:
            result = solve(question, table_id)
            rst,query = results(result,table_id)
            # print(result)
            # print(rst)
            # print(query)
            res = {"msg": "ok", "result": rst, "query": query}
        except:
            res = {"msg": "查询的表不存在！"}
    res = json.dumps(res, ensure_ascii=False)
    # res = res.replace('\\', '')
    # print(res)
    return res

 
if __name__ == '__main__':
    app.config['JSON_AS_ASCII'] = False
    CORS(app, supports_credentials=True)
    app.run(host='0.0.0.0', port=18888)



e:\Anaconda3\envs\nl2sql\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Loaded 4055 data from ../data/final/final_test.json
Loaded 978 data from ../data/final/final_test.tables.json
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:18888/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jun/2020 09:34:33] "POST /nl2sql HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2020 09:37:59] "POST /nl2sql HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2020 09:38:43] "POST /nl2sql HTTP/1.1" 200 -


In [3]:
print(solve("这种BT-1574型号的发声拉是什么公司的产品震玩具", "a88bac7a3b0611e9a29bf40f24344a08"))

NameError: name 'solve' is not defined

In [ ]:
print(torch.__version__)

In [ ]:
model.load_state_dict(torch.load(restore_model_path, map_location=torch.device('cpu')))